In [180]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

data_sets = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        print(filename)
        if filename == "test_essays.csv":
            test = pd.read_csv(os.path.join(dirname, filename))
        if filename == "train_essays.csv":
            train = pd.read_csv(os.path.join(dirname, filename))
        if filename == "train_essays_RDizzl3_seven_v2.csv":
            generated_text = pd.read_csv(os.path.join(dirname, filename))
        data_sets.append(pd.read_csv(os.path.join(dirname, filename)))
    print(f"{len(data_sets)} data sets loaded in.")
    
    
#test = train[10:100]
#sample_sub, training_prompts, test, train, _, _, _, generated_text = data_sets

0 data sets loaded in.
sample_submission.csv
train_prompts.csv
test_essays.csv
train_essays.csv
4 data sets loaded in.
train_essays_RDizzl3_seven_v2.csv
train_essays_7_prompts_v2.csv
train_essays_7_prompts.csv
train_essays_RDizzl3_seven_v1.csv
8 data sets loaded in.


In [181]:
print("Supplementary dataset features:")
print(generated_text.dtypes)
print("\nOriginal dataset features:")
print(train.dtypes)


Supplementary dataset features:
text     object
label     int64
dtype: object

Original dataset features:
id           object
prompt_id     int64
text         object
generated     int64
dtype: object


As we can see, we need to make it so that these share a universal format so that we can combine the data together.

In [182]:
test

,id,prompt_id,text,generated
10,020a5d6d,1,"Dear senator, Retain the Electoral College. Th...",0
11,037352a5,0,"""When I had a car I was always tense. I'm much...",0
12,03a3377a,1,Voting for a president is one of the most impo...,0
13,03c28f3e,1,"dear senator, I have come to a conclusion on h...",0
14,03cabe10,0,Everyone wants to go to the park with their ch...,0
...,...,...,...,...
95,1539183b,0,"In the world we live in today, cars are used b...",0
96,15eaf968,1,When electing the president of the United Stat...,0
97,15f7ea58,1,"Dear Senator, Concerning the topic of the meri...",0
98,162be3e7,0,Sometimes you don't need a car to go some plac...,0


In [183]:
# Rename the label column to match original dataset
generated_text.rename(columns={'label': 'generated'}, inplace=True)
generated_text

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
17246,"Dear Senator,\n\nI am writing to you today to ...",1
17247,"Dear Senator,\n\nI am writing to you today to ...",1
17248,"Dear Senator,\n\nI am writing to you today to ...",1
17249,"Dear Senator,\n\nI am writing to you today to ...",1


In [184]:
# Drop the two columns we don't need for training
train.drop('id', axis='columns', inplace=True)
train.drop('prompt_id', axis='columns', inplace=True)
train

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
1373,There has been a fuss about the Elector Colleg...,0
1374,Limiting car usage has many advantages. Such a...,0
1375,There's a new trend that has been developing f...,0
1376,As we all know cars are a big part of our soci...,0


In [185]:
training_set = pd.concat([generated_text, train])

In [186]:
training_set.describe()

,generated
count,18629.000000
mean,0.161415
std,0.367923
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [187]:
from sklearn.model_selection import train_test_split
import keras_nlp
import keras_core as keras

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
   

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping

In [188]:
isRealTest = True

if isRealTest:
    X_train = training_set['text']
    y_train = training_set['generated']
    
    X_test = test['text']
    X_testIDs = test['id']
    
    
else:
    X = training_set['text']
    y = training_set['generated']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [189]:
max_words = 10000
padding_length = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Tokenize them into vectors
# Each unique word represented by an integer
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad our data to ensure consistent length
X_train_padded = pad_sequences(X_train_sequences, maxlen=padding_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=padding_length, padding='post', truncating='post')

In [190]:
print(X_test)

10    Dear senator, Retain the Electoral College. Th...
11    "When I had a car I was always tense. I'm much...
12    Voting for a president is one of the most impo...
13    dear senator, I have come to a conclusion on h...
14    Everyone wants to go to the park with their ch...
                            ...                        
95    In the world we live in today, cars are used b...
96    When electing the president of the United Stat...
97    Dear Senator, Concerning the topic of the meri...
98    Sometimes you don't need a car to go some plac...
99    Car alarms, car horns, and engines are basical...
Name: text, Length: 90, dtype: object


### Visualizing our preprocessing

Now we can see the three steps our data goes through


In [191]:
X_train[4]

4    Cars are a wonderful thing. They are perhaps o...
4    Cars are a wonderful thing. They are perhaps o...
Name: text, dtype: object

In [192]:
essay_number = 5

print("Step 1: Raw Text\n"+ X_train[essay_number])
print("\n\nStep 2: Tokens\n" + str(X_train_sequences[essay_number]))
print("\n\nStep 3: Padded Set\n" + str(X_train_padded[essay_number]))

print("Generated: " + str(y_train[essay_number]))

5    Step 1: Raw Text\nThe electrol college system ...
5    Step 1: Raw Text\nThe electrol college system ...
Name: text, dtype: object


Step 2: Tokens
[2, 5260, 30, 69, 8, 55, 238, 69, 21, 158, 17, 2, 243, 3, 2002, 44, 221, 51, 19, 250, 17, 2, 243, 3, 2002, 5, 51, 39, 52, 21, 37, 19, 13, 2003, 148, 11, 2, 93, 11, 5, 1, 9, 146, 20, 12, 5, 411, 28, 5, 1602, 7, 139, 239, 10, 38, 9, 93, 20, 402, 36, 2621, 146, 86, 9329, 2, 2, 423, 148, 212, 88, 64, 26, 37, 26, 37, 3, 2002, 5, 1279, 68, 40, 5629, 14, 197, 6, 74, 343, 5, 225, 4, 446, 2, 91, 276, 169, 88, 2, 5260, 1, 8, 238, 39, 4, 2, 5218, 10, 100, 43, 1296, 26, 29, 365, 21, 711, 402, 44, 221, 552, 29, 10, 189, 7, 139, 239, 9, 52, 126, 37, 19, 13, 148, 11, 2, 170, 93, 183, 111, 562, 56, 37, 6, 108, 2, 93, 20, 402, 68, 19, 124, 11, 51, 7, 139, 239, 10, 38, 9, 2, 24, 30, 1175, 4, 931, 93, 6, 2, 91, 229, 4, 24, 76, 8, 1292, 7, 599, 3, 824, 55, 51, 47, 7, 139, 239, 10, 38, 9, 2, 173, 4, 93, 26, 17, 3068, 3, 2, 229, 4, 1666, 4, 484, 26, 17, 27,

In [193]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=200),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss =tf.keras.losses.BinaryCrossentropy(), metrics=[BinaryAccuracy()])

In [194]:
y_train = y_train.astype(np.float32)
if not isRealTest:
    y_test  = y_test .astype(np.float32)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(X_train_padded, y_train, epochs=5, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/5
524/524 [==============================] - 59s 108ms/step - loss: 0.2644 - binary_accuracy: 0.9007 - val_loss: 0.2015 - val_binary_accuracy: 0.9469
Epoch 2/5
524/524 [==============================] - 55s 105ms/step - loss: 0.0997 - binary_accuracy: 0.9868 - val_loss: 0.0848 - val_binary_accuracy: 0.9973
Epoch 3/5
524/524 [==============================] - 55s 106ms/step - loss: 0.0486 - binary_accuracy: 0.9934 - val_loss: 0.0288 - val_binary_accuracy: 0.9979
Epoch 4/5
524/524 [==============================] - 56s 107ms/step - loss: 0.0213 - binary_accuracy: 0.9967 - val_loss: 0.0090 - val_binary_accuracy: 1.0000
Epoch 5/5
524/524 [==============================] - 57s 109ms/step - loss: 0.0143 - binary_accuracy: 0.9979 - val_loss: 0.0059 - val_binary_accuracy: 1.0000


In [195]:
if not isRealTest:
    # Evaluate the model on the test set
    loss, accurtestcy = model.evaluate(X_test_padded, y_test)
    print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

In [196]:
from sklearn.metrics import confusion_matrix, classification_report

# Predictions on the test set
y_pred = model.predict(X_test_padded)

# Convert probabilities to class labels
y_pred_labels = np.round(y_pred)

3/3 [==============================] - 1s 29ms/step


In [211]:
if not isRealTest:
    # Confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_labels)
    print("Confusion Matrix:")
    print(conf_matrix)

    # Classification report
    class_report = classification_report(y_test, y_pred_labels)
    print("\nClassification Report:")
    print(class_report)

if isRealTest:
    submission = pd.DataFrame({'id': list(X_testIDs), 'generated': y_pred.flatten()})

In [212]:
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [213]:
submission

,id,generated
0,020a5d6d,0.001528
1,037352a5,0.001511
2,03a3377a,0.001520
3,03c28f3e,0.001714
4,03cabe10,0.001484
...,...,...
85,1539183b,0.001403
86,15eaf968,0.001558
87,15f7ea58,0.001481
88,162be3e7,0.001404
